# Introduction

This is a notebook that show a few tricks that allow to create language model with fastai, ULMFiT and Colab free GPU for polish wikipedia.

Although fastai works fine on PC with newest python, it would take aproximetely 60 days to create model with CPU only. In Colab python3.6 there are some bugs, and below I show how to make it working with Colab GPU within 20 hours of GPU.

First of all sometimes "Terminate" button is disabled in Colab Jupiter Notebook. Then restart is possible with:

Please see original notebooks from fastai team:
https://github.com/fastai/course-nlp/blob/master/nn-turkish.ipynb

In [0]:
!kill -9 -1

# Data preparation

In [0]:
import os

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


At first running autoreload should be started, but after first data.py code change Restart runtime is required and then reload section should be skipped.

In [0]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [0]:
from fastai import *
from fastai.text import *

In [3]:
torch.cuda.get_device_name(0) #check GPU

'Tesla P100-PCIE-16GB'

In [0]:
data_path = Config.data_path()
lang = 'pl'
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)

mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
lm_fns = [mdl_path/f'{lang}_wt', mdl_path/f'{lang}_wt_vocab']

In [29]:
cd /content/gdrive/My\ Drive/

/content/gdrive/My Drive


In [30]:
os.getcwd()

'/content/gdrive/My Drive'

Here I had some problem wih importing nlputils, so I copied it, adding one more function extract_only which can be helpful. (.. to be imporoved)

In [0]:
from fastai.basics import *
import re


def get_wiki(path,lang):
    name = f'{lang}wiki'
    if (path/name).exists():
        print(f"{path/name} already exists; not downloading")
        return
    
    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    zip_fn = f"{xml_fn}.bz2"
    
    if not (path/xml_fn).exists():
        print("downloading...")
        download_url(f'https://dumps.wikimedia.org/{name}/latest/{zip_fn}', path/zip_fn)
        print("unzipping...")
        bunzip(path/zip_fn)
    
    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def extract_only(path,lang):
    name = f'{lang}wiki'
    xml_fn = f"{lang}wiki-latest-pages-articles.xml"
    with working_directory(path):
        if not (path/'wikiextractor').exists(): os.system('git clone https://github.com/attardi/wikiextractor.git')
        print("extracting...")
        os.system("python wikiextractor/WikiExtractor.py --processes 4 --no_templates " +
            f"--min_text_length 1800 --filter_disambig_pages --log_file log -b 100G -q {xml_fn}")
    shutil.move(str(path/'text/AA/wiki_00'), str(path/name))
    shutil.rmtree(path/'text')


def split_wiki(path,lang):
    dest = path/'docs'
    name = f'{lang}wiki'
    if dest.exists():
        print(f"{dest} already exists; not splitting")
        return dest

    dest.mkdir(exist_ok=True, parents=True)
    title_re = re.compile(rf'<doc id="\d+" url="https://{lang}.wikipedia.org/wiki\?curid=\d+" title="([^"]+)">')
    lines = (path/name).open()
    f=None

    for i,l in enumerate(lines):
        if i%100000 == 0: print(i)
        if l.startswith('<doc id="'):
            try:
                title = title_re.findall(l)[0].replace('/','_').replace(':','_')
            except:
                title = 'title{}'.format(i)
            if len(title)>150: continue
            if f: f.close()
            f = (dest/f'{title}.txt').open('w')
        else: f.write(l)
    f.close()
    return dest



In [38]:
get_wiki(path,lang)
!head -n4 {path}/{name}

downloading...


unzipping...
extracting...
<doc id="2" url="https://pl.wikipedia.org/wiki?curid=2" title="AWK">
AWK

AWK – interpretowany język programowania, którego główną funkcją jest wyszukiwanie i przetwarzanie wzorców w plikach lub strumieniach danych. Jest także nazwą programu początkowo dostępnego dla systemów operacyjnych będących pochodnymi UNIX-a, obecnie także na inne platformy.


In [5]:
!ls /root/.fastai/data/plwiki

docs   models				 plwiki-latest-pages-articles.xml.bz2
doc_s  plwiki				 wikiextractor
log    plwiki-latest-pages-articles.xml


In [43]:
dest = split_wiki(path,lang)

0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000
3200000
3300000
3400000
3500000
3600000
3700000
3800000
3900000
4000000
4100000
4200000
4300000
4400000
4500000
4600000
4700000
4800000
4900000
5000000
5100000
5200000
5300000
5400000
5500000
5600000
5700000
5800000
5900000
6000000
6100000


In [46]:
!ls /root/.fastai/data/plwiki/docs/Zygmunt\ U*

'/root/.fastai/data/plwiki/docs/Zygmunt Ułanowicz.txt'
'/root/.fastai/data/plwiki/docs/Zygmunt Ulm.txt'
'/root/.fastai/data/plwiki/docs/Zygmunt Unrug.txt'
'/root/.fastai/data/plwiki/docs/Zygmunt Urbanyj.txt'


In [47]:
!cat '/root/.fastai/data/plwiki/docs/Zygmunt Unrug.txt'

Zygmunt Unrug

Zygmunt Unrug (ur. 1676 w Międzychodzie, zm. 1 maja 1732) – polski szlachcic z rodu Unrugów, dyplomata, protestant (brat czeski). Absolwent szkoły w Lesznie, następnie Uniwersytetu we Frankfurcie, prześladowany z powodów religijnych.

Urodził się w 1676. Był synem Krzysztofa Unruga (1624-1689) herbu własnego, właściciela Międzychodu i Kargowej, starosty gnieźnieńskiego i wałeckiego oraz Bogumiły Jaskóleckiej herbu Zaremba. Po ojcu odziedziczył Poniec, Kawcze, Szelejewo i Zduny. Ożenił się z Beatą Szarlotą Luizą Gorzeńską herbu Nałęcz. Mieli dwóch synów: Chryzostoma Zygmunta i Krzysztofa Zygmunta (1728-1768), późniejszego generała armii koronnej.

Był pułkownikiem armii koronnej, a od 1698 starostą gnieźnieńskim. W 1705 potwierdził "pacta conventa" Stanisława Leszczyńskiego. Przyczynił się do detronizacji Augusta II Sasa, ujawniając jego tajne listy. W 1708 król Stanisław Leszczyński wysłał go do Berlina by wybadał nastawienie Fryderyka I do Polski i chęć ewentualnej medi

In [0]:
!cp /content/gdrive/My\ Drive/language_model/data/plwiki/plwiki_old /root/.fastai/data/plwiki/

In [10]:
!ls -l /root/.fastai/data/plwiki/doc_s

total 6052
-rw-r--r-- 1 root root 3698372 Jan 18 18:35  pl_databunch
drwxr-xr-x 2 root root    4096 Jan 18 18:34  tmp
-rw-r--r-- 1 root root    2401 Jan 18 18:12 'Zybert II Dobry.txt'
-rw-r--r-- 1 root root    4413 Jan 18 18:12  Zybułtowo.txt
-rw-r--r-- 1 root root   10160 Jan 18 18:12 'Zyfia gęsiogłowa.txt'
-rw-r--r-- 1 root root    2076 Jan 18 18:12  Zyfiowate.txt
-rw-r--r-- 1 root root    2149 Jan 18 18:12 'Zygaena fausta.txt'
-rw-r--r-- 1 root root    4940 Jan 18 18:12 'Zygfryd Berezecki.txt'
-rw-r--r-- 1 root root    4735 Jan 18 18:12 'Zygfryd Czempisz.txt'
-rw-r--r-- 1 root root    2075 Jan 18 18:12 'Zygfryd Goldfinger.txt'
-rw-r--r-- 1 root root    2473 Jan 18 18:12 'Zygfryd Gölis.txt'
-rw-r--r-- 1 root root    3060 Jan 18 18:12 'Zygfryd II z Eppsteinu.txt'
-rw-r--r-- 1 root root    2470 Jan 18 18:12 'Zygfryd I (palatyn reński).txt'
-rw-r--r-- 1 root root    2448 Jan 18 18:12 'Zygfryd Kuchta.txt'
-rw-r--r-- 1 root root    1962 Jan 18 18:12 'Zygfryd Kuliński.txt'
-rw-r--r-- 1 roo

Backup your plwiki to your local folder to avoid downloading, unzipping and extracting. (split_wiki is quite fast so it can be done again)

In [0]:
!cp /root/.fastai/data/plwiki/plwiki /content/gdrive/My\ Drive/

In [0]:
#!head -20 /content/gdrive/My\ Drive/language_model/data/plwiki/plwiki_old

In [0]:
''' I rewrited plwiki with changing coding under another enviroment, but I think it is not needed here
name3 = 'newplwiki'
with open(path/name,'r',encoding='utf-8') as file:
    with open(path/name3,'w',encoding='utf-8') as file2:
        for line in file:
            #line = file.readline()
            try:
                line2 = line.encode('cp1250').decode('utf-8')
                file2.write(line2)
            except:
                words = line2.split(' ')
                for w in words:
                    file2.write(w+' ')
'''

Creating test environmet with small dataset.

In [0]:
mkdir /root/.fastai/data/plwiki/doc_s

In [0]:
!cp /root/.fastai/data/plwiki/docs/Zy* /root/.fastai/data/plwiki/doc_s/

Choose if you would like to work on full data or a sample.

In [0]:
dest = '/root/.fastai/data/plwiki/doc_s'

In [0]:
dest = '/root/.fastai/data/plwiki/docs'

In [0]:
!ls -l /root/.fastai/data/plwiki/doc_s/tmp

total 0


# Model creation

In [8]:
!pip install sentencepiece

After first run you will get an error:

FileNotFoundError: [Errno 2] No such file or directory: '/root/.fastai/data/plwiki/doc_s/tmp/spm/spm.vocab'

Please click on error link /usr/local/lib/python3.6/dist-packages/fastai/text/data.py and do some changes:
In ```def train_sentencepiece``` change from code
```
    cache_dir = cache_dir/'spm'
    SentencePieceTrainer.Train(" ".join([
        f"--input={raw_text_path} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={ifnone(char_coverage, 0.99999 if lang in full_char_coverage_langs else 0.9998)}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f'--model_prefix="cache_dir" --vocab_size={vocab_sz} --model_type={model_type}']))
    raw_text_path.unlink()
    return cache_dir
```
to:
```
    return_cache_dir = cache_dir
    cache_dir = cache_dir/'spm'
    SentencePieceTrainer.Train(" ".join([
        f"--input={raw_text_path} --max_sentence_length={max_sentence_len}",
        f"--character_coverage={ifnone(char_coverage, 0.99999 if lang in full_char_coverage_langs else 0.9998)}",
        f"--unk_id={len(defaults.text_spec_tok)} --pad_id=-1 --bos_id=-1 --eos_id=-1",
        f"--user_defined_symbols={','.join(spec_tokens)}",
        f'--model_prefix={cache_dir} --vocab_size={vocab_sz} --model_type={model_type}']))
    raw_text_path.unlink()
    return return_cache_dir
```

and in ```def process(self, ds):```
change line which starts with: 
```
if self.n_cpus <= 1:
```
to
```
if self.n_cpus <= 4:
```

there restart Runtime, do not execute autoreload section, remmeber to backup plwiki file and restore it





In [4]:
!rm -r /root/.fastai/data/plwiki/doc_s/tmp
bs=128
data = (TextList.from_folder(dest, processor=[OpenFileProcessor(), SPProcessor(lang='pl')])
        .split_by_rand_pct(0.1, seed=42)
        .label_for_lm(ignore_empty=True)
        .databunch(bs=bs))

#print(data)
#data.save('pl_databunch')
len(data.vocab.itos),len(data.train_ds)

(11328, 399)

In [0]:
data.save('pl_databunch')

In [0]:
!cp /root/.fastai/data/plwiki/docs/pl_databunch /content/gdrive/My\ Drive/language_model/data/plwiki/

In [6]:
data.show_batch()

idx,text
0,"▁1906 –19 13 ▁uczył ▁się ▁w ▁gimnazjum ▁w ▁xxmaj ▁orle , ▁ a ▁po ▁uzyskani u ▁świadectw a ▁dojrzałości ▁rozpoczął ▁studia ▁na ▁xxmaj ▁wydziale ▁xxmaj ▁medycznym ▁xxmaj ▁uniwersytetu ▁w ▁xxmaj ▁moskwie . ▁w ▁luty m ▁1915 ▁roku ▁powołany ▁do ▁armii ▁rosyjskiej , ▁gdzie ▁ukończył ▁xxmaj ▁aleksandr owską ▁xxmaj ▁szkołę ▁xxmaj ▁piechoty ▁w ▁xxmaj ▁moskwie . ▁xxmaj ▁następnie , ▁kolejno ▁był ▁dowódcą ▁kompanii ▁w ▁ 33 ▁pułku ▁piechoty , ▁oddziału ▁łączności"
1,"▁xxmaj ▁paryskiej ▁oraz ▁xxmaj ▁ biuletyn u ▁xxmaj ▁dolnośląski ego ▁wydawan ego ▁przez ▁xxmaj ▁kornel a ▁xxmaj ▁morawiecki ego . ▁xxmaj ▁jako ▁członek ▁ solidarności ▁wraz ▁z ▁grupą ▁osób ▁xxmaj ▁ krzysztof ▁xxmaj ▁tenerowicz , ▁xxmaj ▁piotr ▁xxmaj ▁załuski , ▁xxmaj ▁ jarosław ▁xxmaj ▁ szym kiewicz , ▁xxmaj ▁jacenty ▁xxmaj ▁lipiński . ▁w ▁stanie ▁wojennym ▁już ▁od ▁13 ▁grudnia ▁1981 ▁roku ▁brał ▁udział ▁w ▁zorganizowan ej ▁przez ▁xxmaj ▁tadeusza"
2,"▁( zm . ▁w ▁1939 ), ▁małżeństwo ▁było ▁bezdzietne . ▁< ▁/ ▁doc > ▁xxbos ▁xxmaj ▁zygmunt ▁xxmaj ▁pomarański ▁xxmaj ▁zygmunt ▁xxmaj ▁pomarański ▁ps . ▁"" br z ó zka "" ▁( ur . ▁20 ▁stycznia ▁1898 ▁w ▁xxmaj ▁warszawie , ▁zm . ▁29 ▁września ▁1941 ▁w ▁xxmaj ▁auschwitz ) ▁– ▁polski ▁żołnierz ▁xxmaj ▁pierwszej ▁xxmaj ▁kompanii ▁xxmaj ▁kadrowej , ▁kapitan ▁rezerwy ▁xxmaj ▁wojska ▁xxmaj ▁polskiego , ▁komendant ▁xxmaj ▁polskiej"
3,"o - wiedeńskiej ▁xxmaj ▁kolei ▁xxmaj ▁żelaznej . ▁w ▁1908 ▁wstąpił ▁do ▁warszawskiego ▁seminarium ▁duchownego , ▁w ▁1913 ▁przy jął ▁święcenia ▁ su b dia kon atu ▁i ▁został ▁skierowany ▁na ▁studia ▁do ▁xxmaj ▁gr ego rian um , ▁gdzie ▁obroni ł ▁pracę ▁doktorsk ą ▁z ▁prawa ▁kanonicznego . ▁xxmaj ▁święcenia ▁kapłańskie ▁przy jął ▁30 ▁listopada ▁1916 . ▁xxmaj ▁po ▁święcenia ch ▁pracował ▁krótko ▁jako ▁wikariusz ▁w ▁parafii ▁św ."
4,"▁– ▁zgrupowanie , ▁wyjazd ▁na ▁mecz e ▁z ▁xxmaj ▁jugosławi ą ▁przed ▁olimpiad ą ▁w ▁xxmaj ▁montrealu . ▁xxmaj ▁zygmunt ▁xxmaj ▁pacuszka ▁wraz ▁z ▁xxmaj ▁kazimierzem ▁xxmaj ▁ szczerb ą ▁zostali ▁wytypowan i ▁w ▁ wadze ▁ lekkopółśredniej ▁do ▁udziału ▁w ▁olimpiad zie , ▁która ▁od bę dzie ▁się ▁w ▁xxmaj ▁montrealu ▁jesienią . ▁xxmaj ▁do ▁xxmaj ▁belgrad u ▁10 ▁kadr owicz ów ▁udał o ▁się ▁pod ▁ wodzą ▁trenera"


In [0]:
!ls -lh /root/.fastai/data/plwiki/docs/pl*

-rw-r--r-- 1 root root 1.9G Jan 11 20:01 /root/.fastai/data/plwiki/docs/pl_databunch


In [0]:
fastai_local_files = '/content/gdrive/My Drive/language_model/data/plwiki/'

In [0]:
bs = 128
data = load_data(dest,'pl_databunch',bs=bs)

In [0]:
epochs_data = '/content/gdrive/My Drive/language_model/data/plwiki/epochs_data2/model'

In [0]:
from fastai.callbacks import SaveModelCallback

In [0]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.1, wd=0.1, pretrained=False).to_fp16()

In [0]:
lr = 3e-3
lr *= bs/48  # Scale learning rate by batch size

In [17]:
learn.unfreeze()
#learn.fit_one_cycle(10, lr, moms=(0.8,0.7))
learn.fit_one_cycle(10, lr, moms=(0.8,0.7), 
       callbacks=[SaveModelCallback(learn, every='epoch',  
                  monitor='accuracy', name=epochs_data)])

epoch,train_loss,valid_loss,accuracy,time
0,7.169099,6.631194,0.119754,00:09
1,6.608907,5.999218,0.143731,00:09
2,5.925918,5.189696,0.202660,00:09
3,5.276292,4.722189,0.238504,00:09
4,4.760765,4.503273,0.258984,00:09
5,4.345967,4.362858,0.272749,00:09
6,3.981343,4.300931,0.281827,00:09
7,3.595247,4.301994,0.285324,00:09
8,3.205735,4.354615,0.282366,00:09
9,2.907485,4.382166,0.280748,00:09


In [0]:
learn.unfreeze()

In [0]:
learn.fit_one_cycle(10, lr, moms=(0.8,0.7), 
                    start_epoch=8,
                    callbacks=[SaveModelCallback(learn, 
                    every='epoch', monitor='accuracy', 
                    name=epochs_data)])

Loaded /content/gdrive/My Drive/language_model/data/plwiki/epochs_data/model_7


epoch,train_loss,valid_loss,accuracy,time
8,3.614749,3.653196,0.364300,1:58:49
9,3.516606,3.531486,0.380321,1:58:48


In [0]:
learn.to_fp32().save(lm_fns[0], with_opt=False)
learn.data.vocab.save(lm_fns[1].with_suffix('.pkl'))

In [0]:
!ls -l /root/.fastai/data/plwiki/*

-rw-r--r-- 1 root root 129095776 Jan 13 20:49 /root/.fastai/data/plwiki/plfine_tuned_enc.pth

/root/.fastai/data/plwiki/models:
total 126756
-rw-r--r-- 1 root root 129216143 Jan 13 20:40 pl_wt.pth
-rw-r--r-- 1 root root    578894 Jan 13 20:40 pl_wt_vocab.pkl


In [0]:
!cp /root/.fastai/data/plwiki/*.pth /content/gdrive/My\ Drive/language_model/data/plwiki/models/

In [0]:
learn.save_encoder(f'/root/.fastai/data/plwiki/{lang}fine_tuned_enc')

In [0]:
#quality check - does the model predict sensible tokens?
TEXT = "to był jeden"
N_WORDS = 40
N_SENTENCES = 2

In [0]:
!mkdir /root/.fastai/data/plwiki/docs/tmp

In [0]:
!cp /content/gdrive/My\ Drive/language_model/data/plwiki/spm.* /root/.fastai/data/plwiki/docs/tmp

In [23]:
print("\n".join(learn.predict(TEXT, N_WORDS, temperature=0.92) for _ in range(N_SENTENCES)))

to był jeden ▁z ▁najwybitniejszy ch ▁członków ▁xxmaj ▁hieronim a ▁xxmaj ▁bo p su , ▁jednak ▁większość ▁co ▁towarzyszył a ▁mu ▁zwycięstwo , ▁jak ▁i ▁rezydent ▁w ▁xxmaj ▁zaklikow ie . ▁xxmaj ▁ le pas ty k ▁zdobył ▁8 ▁stycznia ▁1981 ▁xxmaj ▁peter
to był jeden ▁z ▁czołowy ch ▁chłopów , ▁martwe ▁naukowe , ▁pasj i ▁i ▁obrońcy . ▁xxmaj ▁na konywał ▁stanowisko ▁zastępcy ▁docenta ▁podobny m ▁w ▁charakterze ▁starszego ▁dyrektora . ▁1 ▁maja ▁1962 , ▁po ▁ o puszczeni u ▁xxup ▁ps p ▁„ piast
